In [ ]:
from datetime import datetime, timedelta
from ecallisto_ng.data_download.downloader import (
    get_ecallisto_data,
    get_instrument_with_available_data,
    get_ecallisto_data_generator,
)
from ecallisto_ng.data_processing.utils import (
    elimwrongchannels,
    subtract_constant_background,
)
from ecallisto_ng.plotting.plotting import plot_spectogram
from datetime import datetime, timedelta

### Get data directly from soleil
Thanks to smart multiprocessing and fast filtering, we can get data directly from the soleil server.
We offer different functions for you to get the data you want.

In [ ]:
start_datetime = datetime(2021, 5, 7, 0, 00, 0)
end_datetime = datetime(2021, 5, 7, 23, 59, 0)

In [ ]:
get_instrument_with_available_data(start_datetime, end_datetime) 

In [ ]:
dfs = get_ecallisto_data(start_datetime, end_datetime, verbose=True)

In [ ]:
df = dfs["Australia-ASSA_01"]

In [ ]:
# Before downsamping
df.shape

In [ ]:
# After downsampling, group time by 1 minute and take the max value
# See here: https://pandas.pydata.org/pandas-docs/dev/reference/api/pandas.DataFrame.resample.html
# The point is here that you cannot plot a lot of rows, so you need to downsample
df.resample("1min").max().shape

In [ ]:
plot_spectogram(df.resample("1min").max())

In [ ]:
# Keep only the channels 40-70. Slice the dataframe
# See: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#slicing-ranges
plot_spectogram(df.loc[:, 40:70].resample("1min").max())

In [ ]:
# Generator, so that each match is downloaded and processed separately
# To allow you to process the data in chunks, save it, etc.
generator = get_ecallisto_data_generator(start_datetime, end_datetime, ['ASSA'])

In [ ]:
for key, value in generator:
    print(key)

In [ ]:
# Groupby to reduce the number of rows and overwrite it
df = df.resample("1min").max()

In [ ]:
# Some spectogram editing
plot_spectogram(elimwrongchannels(subtract_constant_background(df)))

In [ ]:
# Fetch interessting data, from the plot above
df = get_ecallisto_data(
    datetime(2021, 5, 7, 3, 39, 0), datetime(2021, 5, 7, 3, 42, 0), "Australia-ASSA_01"
)

In [ ]:
plot_spectogram(df)

In [ ]:
# Some spectogram editing
plot_spectogram(elimwrongchannels(subtract_constant_background(df)))

In [ ]:
# Make it prettier. .pipe in pandas is a way to chain functions
(df.pipe(elimwrongchannels).pipe(subtract_constant_background).pipe(plot_spectogram))

### Get live data

In [ ]:
now = datetime.now()

start_datetime = now - timedelta(days=1)
end_datetime = now

In [ ]:
df = (
    get_ecallisto_data(
        start_datetime=start_datetime,
        end_datetime=end_datetime,
        instrument_name="AUSTRIA-UNIGRAZ_01",
    )
    .resample("1min")
    .max()
)

In [ ]:
plot_spectogram(df)